In [3]:
from data import VIL, multibatch_collate_fn
from transform import TrainTransform, TestTransform

from options import OPTION as opt

import torch.utils.data as data

input_dim = opt.input_size

train_transformer = TrainTransform(size=input_dim)
test_transformer = TestTransform(size=input_dim)

trainset = VIL( 
    train=True,
    sampled_frames=opt.sampled_frames, # 对每个视频采样9帧
    transform=train_transformer,
    max_skip=opt.max_skip[0], # 帧之间最大跳步,这里取索引0是因为设置列表可将不同采样帧数数据集放一块训练
    samples_per_video=opt.samples_per_video # 对每个视频采样2次，也就是相当于训练2轮
)
testset = VIL( # opt.valset
    train=False,
    transform=test_transformer,
    samples_per_video=1
) # 遍历的时候会出错，kill
trainloader = data.DataLoader(trainset, batch_size=opt.train_batch, shuffle=True, num_workers=opt.workers,collate_fn=multibatch_collate_fn, drop_last=True)
# testloader = data.DataLoader(testset, batch_size=opt.train_batch, shuffle=True, num_workers=opt.workers,collate_fn=multibatch_collate_fn, drop_last=True)
# multibatch_collate_fn 在batch>1的时候，将frames、masks、objs、info分组，batch=1的时候有区别
# for i in trainloader:
#     print(len(i))   #1
#     print(len(i[0]))#4，如果上面不用fn的话，这里还会有一个外包装
#     break
# 如果batch_size=2，没有fn的话是2，4,有fn的时候4，2，更方便

In [6]:
print(len(testloader))
# for frames,masks,obj_num,info in testloader:
#     print(frames.shape)
#     break

20


RuntimeError: DataLoader worker (pid(s) 17338) exited unexpectedly

In [2]:
# VIL-100有100个视频，8:2划分
print(len(trainset)) #480个训练集
for frames,masks,obj_num,info in trainset:
    print(frames.shape)         # torch.Size([9, 3, 240, 427]) 训练集每个视频采样8帧
    print(masks.shape)          # torch.Size([9, 9, 240, 427]) 每个帧有9个mask，8+1背景
    print(obj_num)              # 8 ,全部都是8，最大的
    print(info.keys())          # dict_keys(['name', 'palette', 'size', 'ImgName'])
    break
print(len(trainloader))
for frames,masks,obj_num,info in trainloader:
    print(frames.shape)     # torch.Size([1, 9, 3, 240, 427])
    print(masks.shape)      # torch.Size([1, 9, 9, 240, 427])
    print(obj_num)          # tensor([8])
    print(info[0].keys())   # 是batch个字典，每个dict_keys(['name', 'palette', 'size', 'ImgName'])
    break

160
torch.Size([9, 3, 240, 427])
torch.Size([9, 9, 240, 427])
8
dict_keys(['name', 'palette', 'size', 'ImgName'])
160
torch.Size([1, 9, 3, 240, 427])
torch.Size([1, 9, 9, 240, 427])
tensor([8])
dict_keys(['name', 'palette', 'size', 'ImgName'])


In [4]:
import yaml
with open('../dataset/VIL100/data/db_info.yaml', 'r') as f:
    db = yaml.load(f, Loader=yaml.Loader)['sequences']

    targetset = 'train'
    # targetset = 'training'
    info = db
    videos = [info['name'] for info in db if info['set'] == targetset]
    videos1 = [info['name'] for info in db if info['set'] == 'test']
    print(len(videos))
    print(len(videos1))

80
20
